# Setup

Dieses Notebook dient der explorativen Datenanalyse der Umfrage zur Nutzung von Generativer KI im Unternehmenskontext.
Bitte stelle sicher, dass die Datei `results-survey.csv` im gleichen Verzeichnis liegt und korrekt formatiert ist.

In [2]:
# Pakete importieren
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Anzeigeoptionen
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')

In [3]:
# CSV-Datei laden (Dateiname ggf. anpassen)
file_path = 'results-survey.csv'
df = pd.read_csv(file_path)
df.head()

,Antwort ID,Datum Abgeschickt,Letzte Seite,Start-Sprache,Zufallsgeneratorstartwert,Datum gestartet,Datum letzte Aktivität,Bitte geben Sie Ihren Nachnamen an:,Bitte geben Sie Ihren Vornamen an:,Bitte geben Sie Ihr Geschlecht an:,Bitte geben Sie Ihre Altersgruppe an:,Bitte geben Sie Ihre Studenten E-Mail Adresse ein:,Wie häufig nutzen Sie aktuell generative KI im Arbeitskontext? [Benutzung KI],Kommentar,"Wo setzen Sie Generative KI typischerweise ein? [Texterstellung und -bearbeitung (z.B. Verfassung von Mails, Artikel oder Zusammenfassungen)]","Wo setzen Sie Generative KI typischerweise ein? [Kreative Inhalte erstellen (z.B. Bilder, Musik, Videos, Designs etc.)]","Wo setzen Sie Generative KI typischerweise ein? [Programmieren und Automatisieren (z.B. Code schreiben, Fehler beheben etc.)]","Wo setzen Sie Generative KI typischerweise ein? [Lernen und Bildung (z.B. individuelle Erklärungen, Zusammenfassungen, Lernhilfe etc.)]","Wo setzen Sie Generative KI typischerweise ein? [Kommunikation und Übersetzung verbessern (z.B. Korrektur von Texten, Übersetzungen von Artikeln etc.)]",Kommentar.1,Welche Erwartungen hatten Sie an die Generative KI vor der Nutzung? [Effizienz],Welche Erwartungen hatten Sie an die Generative KI vor der Nutzung? [Kreativität],Welche Erwartungen hatten Sie an die Generative KI vor der Nutzung? [Arbeitsentlastung],Welche Erwartungen hatten Sie an die Generative KI vor der Nutzung? [Zuverlässigkeit],Welche Erwartungen hatten Sie an die Generative KI vor der Nutzung? [Innovation],Kommentar.2,Welche dieser Erwartungen wurden erfüllt? [Effizienz][Skala 1],Welche dieser Erwartungen wurden erfüllt? [Effizienz][Skala 2],Welche dieser Erwartungen wurden erfüllt? [Kreativität][Skala 1],Welche dieser Erwartungen wurden erfüllt? [Kreativität][Skala 2],Welche dieser Erwartungen wurden erfüllt? [Arbeitsentlastung][Skala 1],Welche dieser Erwartungen wurden erfüllt? [Arbeitsentlastung][Skala 2],Welche dieser Erwartungen wurden erfüllt? [Zuverlässigkeit][Skala 1],Welche dieser Erwartungen wurden erfüllt? [Zuverlässigkeit][Skala 2],Welche dieser Erwartungen wurden erfüllt? [Innovation][Skala 1],Welche dieser Erwartungen wurden erfüllt? [Innovation][Skala 2],Kommentar.3,Welche positiven Erfahrungen habe Sie mit Generativer KI gemacht? [Zeitersparnis],Welche positiven Erfahrungen habe Sie mit Generativer KI gemacht? [Kreative Impulse],Welche positiven Erfahrungen habe Sie mit Generativer KI gemacht? [Entlastung (Zeit + Aufwand)],Welche positiven Erfahrungen habe Sie mit Generativer KI gemacht? [Qualität],Welche positiven Erfahrungen habe Sie mit Generativer KI gemacht? [Keine bemerkbaren Effekte],Kommentar.4,Welche negativen Erfahrungen haben Sie mit Generativer KI gemacht? [Ungenaue Resultate],Welche negativen Erfahrungen haben Sie mit Generativer KI gemacht? [Fehlender Datenschutz],Welche negativen Erfahrungen haben Sie mit Generativer KI gemacht? [Verbreitung von Fehlinformationen (Fake News)],Welche negativen Erfahrungen haben Sie mit Generativer KI gemacht? [Abhängigkeit von Technologie],Welche negativen Erfahrungen haben Sie mit Generativer KI gemacht? [Verlust von Originalität / menschlicher Kreativität],Kommentar.5,Wie stark vertrauen Sie den Resultaten von Generativer KI?,Kommentar.6,Hat Generative KI Ihre Arbeitsweise verändert?,Kommentar.7,Wie kompetent fühlen Sie sich im Umgang mit Generativer KI?,Kommentar.8,Hatten Sie Schulungen oder Trainings im Zusammenhang mit Generativer KI?,Kommentar.9,Wie gut unterstützt Ihr Unternehmen die Nutzung von Generativer KI? [Technologische Infrastruktur],Wie gut unterstützt Ihr Unternehmen die Nutzung von Generativer KI? [Guidelines zur Benutzung von Generativer KI],Wie gut unterstützt Ihr Unternehmen die Nutzung von Generativer KI? [Schulungen / Kurse / Webinare],Wie gut unterstützt Ihr Unternehmen die Nutzung von Generativer KI? [Rechtliche Richtlinien],Wie gut unterstützt Ihr Unternehmen die Nutzung von Generativer KI? [Kulturelle Offenheit],Kommentar.10,Würden Sie Generative KI

## Überblick über die Datenstruktur

In [3]:
# Struktur anzeigen, fehlende Werte analysieren
df.info()
df.isnull().sum().sort_values(ascending=False).head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 67 columns):
 #   Column                                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                                   --------------  ----- 
 0   Antwort ID                                                                                                                                               47 non-null     int64 
 1   Datum Abgeschickt                                                                                                                                        47 non-null     object
 2   Letzte Seite                                                                                                                                             47 non-null    

Kommentar.8     45
Kommentar.12    44
Kommentar.11    44
Kommentar.9     44
Kommentar.10    41
Kommentar.5     41
Kommentar.4     40
Kommentar.2     40
Kommentar.1     39
Kommentar.7     39
dtype: int64

In [12]:
print(df.columns.tolist())

['Antwort ID', 'Datum Abgeschickt', 'Letzte Seite', 'Start-Sprache', 'Zufallsgeneratorstartwert', 'Datum gestartet', 'Datum letzte Aktivität', 'Bitte geben Sie Ihren Nachnamen an:', 'Bitte geben Sie Ihren Vornamen an:', 'Bitte geben Sie Ihr Geschlecht an:', 'Bitte geben Sie Ihre Altersgruppe an:', 'Bitte geben Sie Ihre Studenten E-Mail Adresse ein:', 'Wie häufig nutzen Sie aktuell generative KI im Arbeitskontext? [Benutzung KI]', 'Kommentar', 'Wo setzen Sie Generative KI typischerweise ein? [Texterstellung und -bearbeitung (z.B. Verfassung von Mails, Artikel oder Zusammenfassungen)]', 'Wo setzen Sie Generative KI typischerweise ein? [Kreative Inhalte erstellen (z.B. Bilder, Musik, Videos, Designs etc.)]', 'Wo setzen Sie Generative KI typischerweise ein? [Programmieren und Automatisieren (z.B. Code schreiben, Fehler beheben etc.)]', 'Wo setzen Sie Generative KI typischerweise ein? [Lernen und Bildung (z.B. individuelle Erklärungen, Zusammenfassungen, Lernhilfe etc.)]', 'Wo setzen Sie Ge

# Gruppendefinitionen

In [4]:
# Gruppierungsdefinitionen
gruppierungen = {
    "ZHAW_vs_NichtZHAW": {
        "label": "ZHAW-Studenten vs. Arbeitnehmende",
        "column": "Bitte geben Sie Ihre Studenten E-Mail Adresse ein:",
        "mapping": {
            "ZHAW-Studenten": lambda x: isinstance(x, str) and x.endswith("@students.zhaw.ch"),
            "Arbeitnehmende": lambda x: not isinstance(x, str) or not x.endswith("@students.zhaw.ch")
        }
    },
    "Altersgruppe": {
        "label": "Altersgruppen",
        "column": "Bitte geben Sie Ihre Altersgruppe an:"
    },
    "Geschlecht": {
        "label": "Geschlecht",
        "column": "Bitte geben Sie Ihr Geschlecht an:"
    }
}